<a href="https://colab.research.google.com/github/ReutDanino/Semantic-segmentation-mini-project/blob/main/%D7%A8%D7%A9%D7%AA_%D7%97%D7%93%D7%A9%D7%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras

In [1]:
pip install keras==<version>

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `pip install keras==<version>'


In [2]:
pip install --upgrade keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [3]:
import keras
from keras import layers
from keras import ops

import os
import numpy as np
from glob import glob
import cv2
from scipy.io import loadmat
import matplotlib.pyplot as plt

# For data preprocessing
from tensorflow import image as tf_image
from tensorflow import data as tf_data
from tensorflow import io as tf_io
import glob

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import tensorflow as tf
from keras import layers
import numpy as np
from tensorflow import data as tf_data
from tensorflow import io as tf_io
from tensorflow import image as tf_image
import matplotlib.pyplot as plt
import glob

# הגדרת משתנים
IMAGE_SIZE = 400
BATCH_SIZE = 4
NUM_CLASSES = 1
NUM_TRAIN_IMAGES = 200
NUM_VAL_IMAGES = 30

# קריאת קבצים
image_paths = glob.glob("/content/drive/MyDrive/לימודים/מיני פרויקט/תמונות מעובדות/train/CXR_png/*.png")
image_paths.sort()
image_paths = image_paths[0:200]

mask_paths = glob.glob("/content/drive/MyDrive/לימודים/מיני פרויקט/תמונות מעובדות/train/masks/*.png")
mask_paths.sort()
mask_paths = mask_paths[0:200]

image_val_paths = glob.glob("/content/drive/MyDrive/לימודים/מיני פרויקט/תמונות מעובדות/validation/CXR_png/*.png")
mask_val_paths = glob.glob("/content/drive/MyDrive/לימודים/מיני פרויקט/תמונות מעובדות/validation/masks/*.png")
image_val_paths.sort()
mask_val_paths.sort()
image_val_paths = image_val_paths[0:20]
mask_val_paths = mask_val_paths[0:20]

# קריאת תמונה עם נורמליזציה
def read_image(image_path, mask=False):
    image = tf_io.read_file(image_path)
    if mask:
        image = tf_image.decode_png(image, channels=1)
        image.set_shape([None, None, 1])
        image = tf_image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = image / 255.0  # נורמליזציה
        image = tf.where(image > 0, 1.0, 0.0)  # Thresholding
    else:
        image = tf_image.decode_png(image, channels=3)
        image.set_shape([None, None, 3])
        image = tf_image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = image / 255.0  # נורמליזציה
    return image

# בדיקת ערכים לא תקינים
def validate_image(image):
    return not tf.reduce_any(tf.math.is_nan(image)) and not tf.reduce_any(tf.math.is_inf(image))

def load_data(image_path, mask_path):
    image = read_image(image_path)
    mask = read_image(mask_path, mask=True)

    tf.debugging.assert_equal(validate_image(image), True, message="Image contains NaN or infinite values")
    tf.debugging.assert_equal(validate_image(mask), True, message="Mask contains NaN or infinite values")

    print("Loaded image shape:", image.shape)
    print("Loaded mask shape:", mask.shape)

    return image, mask


def data_generator(image_list, mask_list):
    dataset = tf_data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf_data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

train_dataset = data_generator(image_paths, mask_paths)
val_dataset = data_generator(image_val_paths, mask_val_paths)

print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)


Loaded image shape: (400, 400, 3)
Loaded mask shape: (400, 400, 1)
Loaded image shape: (400, 400, 3)
Loaded mask shape: (400, 400, 1)
Train Dataset: <_BatchDataset element_spec=(TensorSpec(shape=(4, 400, 400, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4, 400, 400, 1), dtype=tf.float32, name=None))>
Val Dataset: <_BatchDataset element_spec=(TensorSpec(shape=(4, 400, 400, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4, 400, 400, 1), dtype=tf.float32, name=None))>


In [7]:
# Building the DeepLabV3+ model
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return ops.nn.relu(x)

def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]),
        interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    preprocessed = keras.applications.resnet50.preprocess_input(model_input)
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=preprocessed
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)

In [8]:
model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 400, 400, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item (GetItem)        │ (None, 400, 400)       │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_1 (GetItem)      │ (None, 400, 400)       │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_2 (GetItem)      │ (None, 400, 400)       │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stack (Stack)             │ (None, 400, 400, 3)    │              0 │ get_item[0][0],        │
│                           │                        │                │ get_item_1[0][0],      │
│                           │                        │                │ get_item_2[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 400, 400, 3)    │              0 │ stack[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 406, 406, 3)    │              0 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 200, 200, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 200, 200, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 200, 200, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 202, 202, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 100, 100, 64)   │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 100, 100, 64)   │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 100, 100, 64)   │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 100, 100, 64)   │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 100, 100, 64)   │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 11,852,353 (45.21 MB)

 Trainable params: 11,819,617 (45.09 MB)

 Non-trainable params: 32,736 (127.88 KB)

In [11]:
!pip install keras_unet

In [15]:
from keras_unet.metrics import iou, iou_thresholded

# Training
#loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.1e-4),
    loss='binary_crossentropy',
    metrics=[iou, iou_thresholded]
)

model.fit(train_dataset, validation_data=val_dataset, epochs=50)

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 68s 362ms/step - iou: 0.9680 - iou_thresholded: 0.7090 - loss: 0.2748 - val_iou: 0.1434 - val_iou_thresholded: 0.0673 - val_loss: 2.9720
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 288ms/step - iou: 11.4498 - iou_thresholded: 0.7381 - loss: 0.2455 - val_iou: -0.1059 - val_iou_thresholded: 0.1487 - val_loss: 2.6037
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 282ms/step - iou: -9.5642 - iou_thresholded: 0.7464 - loss: 0.2311 - val_iou: 0.4916 - val_iou_thresholded: 0.1088 - val_loss: 3.2915
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 277ms/step - iou: 0.3653 - iou_thresholded: 0.7521 - loss: 0.2353 - val_iou: -3.2265 - val_iou_thresholded: 0.1968 - val_loss: 2.5237
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 14s 282ms/step - iou: -76.6934 - iou_thresholded: 0.7781 - loss: 0.2003 - val_iou: 0.6226 - val_iou_thresholded: 0.0403 - val_loss: 3.5363
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 278ms/step - iou: -4.3724 - iou_thresholded: 0.7871 - loss: 0.1776 - val_iou:

In [16]:
model.save('/content/drive/MyDrive/לימודים/מיני פרויקט בביו /myNew_model.h5')

In [ ]:
history=model.fit(train_dataset, validation_data=val_dataset, epochs=25)

Epoch 1/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 14s 284ms/step - iou: -1.6100 - iou_thresholded: 0.8814 - loss: 0.0760 - val_iou: -1.9626 - val_iou_thresholded: 0.7978 - val_loss: 0.2945
Epoch 2/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 279ms/step - iou: -1.5821 - iou_thresholded: 0.8825 - loss: 0.0742 - val_iou: -4.6168 - val_iou_thresholded: 0.7950 - val_loss: 0.2446
Epoch 3/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 279ms/step - iou: -1.5592 - iou_thresholded: 0.8820 - loss: 0.0753 - val_iou: -2.8550 - val_iou_thresholded: 0.8041 - val_loss: 0.2466
Epoch 4/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 280ms/step - iou: -1.6608 - iou_thresholded: 0.8642 - loss: 0.1061 - val_iou: 0.5581 - val_iou_thresholded: 0.0024 - val_loss: 4.4432
Epoch 5/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 282ms/step - iou: -1.6987 - iou_thresholded: 0.8522 - loss: 0.1152 - val_iou: 0.4408 - val_iou_thresholded: 1.4061e-04 - val_loss: 4.2926
Epoch 6/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 280ms/step - iou: -1.6616 - iou_thresholded: 0.8690 - loss: 0.0913 - va

In [ ]:
!pip install keras_unet

In [ ]:
plt.plot(history.history["loss"])
plt.title("Training Loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["accuracy"])
plt.title("Training Accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_loss"])
plt.title("Validation Loss")
plt.ylabel("val_loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_accuracy"])
plt.title("Validation Accuracy")
plt.ylabel("val_accuracy")
plt.xlabel("epoch")
plt.show()

In [ ]:
from keras.models import load_model
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from keras_unet.metrics import iou, iou_thresholded



# פונקציה לטעינת ועיבוד התמונה
def load_and_preprocess_image(image_path, target_size):
    img = Image.open(image_path).convert("RGB").resize(target_size)
    img_np = np.array(img, dtype=np.float32) / 255.0
    return img_np

# פונקציה לחיזוי ולהצגת התוצאות
def predict_and_visualize(image_path, target_size, model):
    # טעינת ועיבוד התמונה
    input_image = load_and_preprocess_image(image_path, target_size)

    # הוספת מימד כדי להתאים את התמונה לכניסה של המודל
    input_image_expanded = np.expand_dims(input_image, axis=0)

    # חיזוי המסיכה
    predicted_mask = model.predict(input_image_expanded)[0]

    # הדפסת ערכי התמונה והמסיכה לבדיקה
    print(f"Input image shape: {input_image.shape}")
    print(f"Input image values (min, max): {input_image.min(), input_image.max()}")
    print(f"Predicted mask shape: {predicted_mask.shape}")
    print(f"Predicted mask values (min, max): {predicted_mask.min(), predicted_mask.max()}")

    # הצגת המסיכה החזויה לפני בינאריזציה
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(predicted_mask, cmap='gray')
    plt.title('Predicted Mask (Before Binarization)')
    plt.axis('off')

    # בינאריזציה של המסיכה החזויה
    threshold = 0.5
    predicted_mask_binary = (predicted_mask > threshold).astype(np.uint8)

    # הדפסת ערכי המסיכה הבינארית לבדיקה
    print(f"Binary mask values (unique): {np.unique(predicted_mask_binary)}")

    # הצגת התמונה המקורית
    plt.subplot(1, 2, 2)
    plt.imshow(input_image)
    plt.title('Original Image')
    plt.axis('off')

    # הצגת המסיכה החזויה לאחר בינאריזציה
    plt.figure(figsize=(10, 5))
    plt.imshow(predicted_mask_binary, cmap='gray')
    plt.title('Predicted Mask (After Binarization)')
    plt.axis('off')

    plt.show()


#model_path = '/content/drive/MyDrive/my_model.h5'
#model = load_model(model_path, custom_objects={'iou': iou, 'iou_thresholded': iou_thresholded})

# דוגמה לשימוש בפונקציה
image_path = "/content/drive/MyDrive/לימודים/מיני פרויקט/תמונות מעובדות/test מחולק/CXR_png/CHNCXR_0530_1.png"  # החלף בנתיב המדויק לתמונת הבדיקה שלך
target_size = (400, 400)  # הגדרת גודל התמונה הרצוי בהתאם למודל
predict_and_visualize(image_path, target_size, model)
